In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from skimage.io import imread, imshow
from skimage.transform import resize

import tensorflow as tf
import gc

import pickle
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input , Dense, Flatten, Dropout, Conv2D, MaxPool2D

from tensorflow.keras.applications import inception_resnet_v2

import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import TopKCategoricalAccuracy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
## Load InceptionResnet that we will use for transfer learning

resnet = inception_resnet_v2.InceptionResNetV2()

225222656/225209952 [==============================] - 7s 0us/step


In [4]:
## Constants

IMG_LEN = 299
IMG_SHAPE = (IMG_LEN, IMG_LEN, 3)
N_BREEDS = 60

In [5]:
opener = open(f'drive/MyDrive/1000_pics.pickle', 'rb')
images = pickle.load(opener)
opener.close()

In [6]:
opener = open(f'drive/MyDrive/1000_labels.pickle', 'rb')
labels = pickle.load(opener)
opener.close()

In [7]:
resnet = Model(inputs = resnet.layers[0].input, outputs = resnet.layers[780].output)

In [8]:
for layer in resnet.layers:
  layer.trainable = False

In [9]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(299, 
                                                              299,
                                                              3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [10]:
custom_model = Sequential([
                           data_augmentation,
                           resnet,
                           Dense(N_BREEDS, activation = 'softmax')
])

In [11]:
custom_model.compile(optimizer = Adam(learning_rate = 0.00001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [12]:
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 7)

In [13]:
gc.collect()

29384

In [14]:
!kill process_id

/bin/bash: line 0: kill: process_id: arguments must be process or job IDs


In [15]:
perm = tf.random.shuffle(tf.range(tf.shape(images)[0]))

In [16]:
images = tf.gather(images, perm, axis=0)
labels = tf.gather(labels, perm, axis=0)

In [17]:
history = custom_model.fit(images, labels, validation_split = 0.2, shuffle = True, epochs = 500 , callbacks = [early_stopping])

Epoch 1/500
25/25 [==============================] - 35s 576ms/step - loss: 4.2342 - accuracy: 0.0025 - val_loss: 4.1669 - val_accuracy: 0.0050
Epoch 2/500
25/25 [==============================] - 10s 410ms/step - loss: 4.1947 - accuracy: 0.0063 - val_loss: 4.1406 - val_accuracy: 0.0050
Epoch 3/500
25/25 [==============================] - 10s 413ms/step - loss: 4.1749 - accuracy: 0.0063 - val_loss: 4.1140 - val_accuracy: 0.0150
Epoch 4/500
25/25 [==============================] - 10s 419ms/step - loss: 4.1322 - accuracy: 0.0113 - val_loss: 4.0878 - val_accuracy: 0.0150
Epoch 5/500
25/25 [==============================] - 10s 421ms/step - loss: 4.1066 - accuracy: 0.0175 - val_loss: 4.0623 - val_accuracy: 0.0200
Epoch 6/500
25/25 [==============================] - 11s 426ms/step - loss: 4.0739 - accuracy: 0.0175 - val_loss: 4.0365 - val_accuracy: 0.0250
Epoch 7/500
25/25 [==============================] - 11s 429ms/step - loss: 4.0464 - accuracy: 0.0312 - val_loss: 4.0109 - val_accuracy:

In [18]:
custom_model.save('drive/MyDrive/multi_label_model')

INFO:tensorflow:Assets written to: drive/MyDrive/multi_label_model/assets


In [21]:
custom_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 299, 299, 3)       0         
                                                                 
 model (Functional)          (None, 1536)              54336736  
                                                                 
 dense (Dense)               (None, 60)                92220     
                                                                 
Total params: 54,428,956
Trainable params: 92,220
Non-trainable params: 54,336,736
_________________________________________________________________
